# Sweep Transfer Attack

9.29.21

#### Summary:
- Made each client (7 options) adversarial and perform transfer attacks throughout the system

In [2]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Relevant Libraries
Take it from the run_experiment.py folder

In [3]:
import sys
sys.path.append("../")

# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *

### Generate Aggregator Pre-requisite
- Clients, Test Clients, Ensemble_Learner
- Follow through the code in run_experiment.py

In [4]:
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedAvg"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 1
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar10/FedEM/'
args_.validation = False
args_.tune_steps = 0
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


AssertionError: Download cifar10 dataset!!

In [5]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]
    
# Determine here the different weights of models we desire to compare
model_weights = []

model_weights += [(1,0,0),(0,1,0),(0,0,1)] #single mixture
model_weights += [(0.5,0.5,0),(0.5,0,0.5),(0,0.5,0.5)] # half of 2 mixtures of 3
model_weights += [(1/3,1/3,1/3)] # Equally balanced from 3 mixtures

# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

### Generate Data 

In [6]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [7]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

### Set Up Transfer Attack Scenario
- Set up order of which attacks will take place -- keep the same transferer, but simply swap out the weights of the adversary and flush out the existing data points analysis was done on
- Make dictionaries beforehand recording all of the metrics

In [8]:
# Set Up Dictionaries -- list holds the adversary idx
logs_adv = []

for i in range(len(model_weights)):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities'] = None
    adv_dict['orig_target_hit'] = None
    adv_dict['adv_target_hit'] = None
    adv_dict['metric_variance'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['metric_ingrad'] = None
    logs_adv += [adv_dict]

In [44]:
# Make transferer and Assign model index
victim_idxs = [0,1,2,3,4,5,6]

for adv_idx in victim_idxs:
    print("id", adv_idx)
    # Perform Attack
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    t1.generate_victims(victim_idxs)
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "ifsgm")
    t1.send_to_victims(victim_idxs)
    t1.check_empirical_metrics(orig_flag = True)
    t1.check_empirical_metrics()
    
    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = t1.orig_acc_transfers
    logs_adv[adv_idx]['orig_similarities'] = t1.orig_similarities
    logs_adv[adv_idx]['adv_acc_transfers'] = t1.adv_acc_transfers
    logs_adv[adv_idx]['adv_similarities'] = t1.adv_similarities
    logs_adv[adv_idx]['orig_target_hit'] = t1.orig_target_hit
    logs_adv[adv_idx]['adv_target_hit'] = t1.adv_target_hit
    
    logs_adv[adv_idx]['metric_variance'] = t1.metric_variance
    logs_adv[adv_idx]['metric_alignment'] = t1.metric_alignment
    logs_adv[adv_idx]['metric_ingrad'] = t1.metric_ingrad

id 0
id 1
id 2
id 3
id 4
id 5
id 6


### Print and Record Results
- Organize the results into a matrix and print them into an excel sheet
- Following Categories (sim_benign, sim_adv, target_adv,grad_alignment)

In [45]:
metrics = ['orig_similarities','adv_similarities','adv_target_hit','metric_alignment']

sim_benign = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
target_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
grad_align = np.zeros([len(victim_idxs),len(victim_idxs)]) 

In [46]:
for adv_idx in victim_idxs:
    for victim in victim_idxs:
        sim_benign[adv_idx,victim] = logs_adv[adv_idx][metrics[0]][victim].data.tolist()
        sim_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[1]][victim].data.tolist()
        target_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[2]][victim].data.tolist()
        grad_align[adv_idx,victim] = logs_adv[adv_idx][metrics[3]][victim].data.tolist()

In [53]:
# Save to Excel file

## convert your array into a dataframe
sim_benign_df = pd.DataFrame(sim_benign)
sim_adv_df = pd.DataFrame(sim_adv)
target_adv_df = pd.DataFrame(target_adv)
grad_align_df = pd.DataFrame(grad_align)

## save to xlsx file

#filepath = 'my_excel_file.xlsx'

# df.to_excel(filepath, index=False)

In [54]:
sim_benign_df

0      1      2      3      4      5      6
0  1.000  0.646  0.688  0.800  0.816  0.678  0.778
1  0.654  1.000  0.708  0.810  0.724  0.820  0.814
2  0.690  0.698  1.000  0.700  0.854  0.818  0.808
3  0.784  0.806  0.746  1.000  0.812  0.804  0.898
4  0.840  0.716  0.826  0.860  1.000  0.782  0.874
5  0.652  0.836  0.810  0.802  0.780  1.000  0.900
6  0.722  0.758  0.818  0.866  0.848  0.874  1.000

In [55]:
sim_adv_df

0      1      2      3      4      5      6
0  1.000  0.376  0.574  0.656  0.718  0.560  0.674
1  0.382  1.000  0.654  0.732  0.586  0.814  0.754
2  0.252  0.342  1.000  0.350  0.650  0.734  0.606
3  0.610  0.724  0.670  1.000  0.724  0.794  0.892
4  0.548  0.428  0.892  0.612  1.000  0.776  0.856
5  0.338  0.746  0.908  0.670  0.770  1.000  0.918
6  0.468  0.648  0.884  0.822  0.892  0.930  1.000

In [56]:
target_adv_df

0      1      2      3      4      5      6
0  0.724  0.212  0.396  0.444  0.510  0.374  0.470
1  0.166  0.718  0.440  0.474  0.358  0.560  0.502
2  0.172  0.260  0.906  0.262  0.562  0.646  0.520
3  0.400  0.514  0.494  0.734  0.526  0.584  0.662
4  0.404  0.312  0.774  0.476  0.834  0.658  0.714
5  0.230  0.594  0.804  0.536  0.652  0.846  0.768
6  0.342  0.512  0.756  0.672  0.756  0.802  0.840

In [57]:
grad_align_df

0         1         2         3         4         5         6
0  0.000000  1.383450  1.410900  1.130475  1.158802  1.346276  1.209258
1  1.350262  0.000000  1.359967  1.088210  1.305977  1.097471  1.164444
2  1.363993  1.362397  0.000000  1.317931  1.136333  1.122287  1.197375
3  1.119090  1.092623  1.342348  0.000000  1.133559  1.122916  0.919445
4  1.151435  1.323326  1.157059  1.126639  0.000000  1.107345  0.904484
5  1.299838  1.101603  1.122293  1.100479  1.087580  0.000000  0.872776
6  1.195205  1.174036  1.206898  0.922593  0.909786  0.888685  0.000000